<a href="https://colab.research.google.com/github/LeeJaeEun0/st1220/blob/main/221220_ch04_ML_04_NaiceBayes_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 - 영화리뷰 감성 분석하기
* 긍정, 부정과 관련된 분석

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# https://github.com/e9t/nsmc/
# https://github.com/e9t/nsmc/blob/master/README.md
file_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt'
df = pd.read_csv(file_url, sep='\t', index_col=0)
df.head() # 20만개

,document,label
id,,
8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [6]:
df.tail()

,document,label
id,,
8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
3302770,쓰.레.기,0
5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0
8548411,포풍저그가나가신다영차영차영차,0


In [5]:
df.info() # 결측치 - 8개

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 8112052 to 8548411
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  199992 non-null  object
 1   label     200000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [7]:
df2 = df.dropna() # 결측치를 모두 삭제해주는 기능
df2

,document,label
id,,
8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...
8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
3302770,쓰.레.기,0
5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0


In [9]:
# 너무 큰 데이터 - 위에서 1000개, 아래에서 1000개
# 같은 모양의 df를 연결할때 pd.concat
df3 = pd.concat([
    df2.head(1000), # 행단위 -> default(axis = 0),

    df2.tail(1000)
])
df3

,document,label
id,,
8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...
8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
3302770,쓰.레.기,0
5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0


## 전처리

In [10]:
df3.document.iloc[0] # 한글 데이터 전처리시 문제가 있음
# 어릴 때 보고 지금 다시 봐도 재밌어요ㅋㅋ
# 어리다 -> 어릴, 재미+나다 => 구성이 복잡함
# 한글 자연어 분석 -> 형태소(언어의 최소 의미단위)로 분석해야함
# 형태소 분석기

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [11]:
!pip install konlpy --quiet
# https://konlpy.org/ko/latest/index.html
# 문장을 원하는 품사로 잘라줌

     |████████████████████████████████| 19.4 MB 623 kB/s 
     |████████████████████████████████| 465 kB 50.3 MB/s 


In [12]:
from konlpy.tag import Okt

In [13]:
okt = Okt()

In [15]:
sample_text = df3.document.iloc[0]
sample_text

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [16]:
okt.nouns(sample_text) # 명사만 추출

['때', '보고', '지금', '다시']

In [19]:
df3['document'] # 명사로 분리되기 전

id
8112052                                   어릴때보고 지금다시봐도 재밌어요ㅋㅋ
8132799     디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...
4655635                  폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
9251303     와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
10067386                          안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
                                  ...                        
8963373                                        포켓 몬스터 짜가 ㅡㅡ;;
3302770                                                 쓰.레.기
5458175                     완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.
6908648                   왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ
8548411                                       포풍저그가나가신다영차영차영차
Name: document, Length: 2000, dtype: object

In [20]:
df3['nouns'] = df3.document.apply(okt.nouns)
df3['nouns'] # 단어들 위주로 분류

id
8112052                                       [때, 보고, 지금, 다시]
8132799     [디자인, 학생, 외국, 디자이너, 그, 일군, 전통, 통해, 발전, 문화, 산업,...
4655635                          [폴리스스토리, 시리즈, 부터, 뉴, 하나, 최고]
9251303                           [연기, 진짜, 개, 생각, 몰입, 진짜, 영화]
10067386                                   [안개, 밤하늘, 초승달, 영화]
                                  ...                        
8963373                                             [포켓, 몬스터]
3302770                                                [레, 기]
5458175              [완전, 사이코, 영화, 마지막, 더욱더, 이, 영화, 질, 트, 린다]
6908648                                    [왜, 라따뚜이, 스머프, 런가]
8548411                                   [포, 저그, 영차, 영차, 영차]
Name: nouns, Length: 2000, dtype: object

In [24]:
df3['nouns'] = df3.document.apply(okt.nouns).apply(
    # nouns : okt.nouns로 분석된 명사들 리스트
    lambda nouns: [n for n in nouns if len(n) >= 2] # len(텍스트) => 텍스트의 길이
    # 텍스트가 2이상인 n만 남김 -> 2글자 이상의 글자만 남음
) # 2글자 미만의 명사들은 모두 제외
df3['nouns']

id
8112052                                          [보고, 지금, 다시]
8132799     [디자인, 학생, 외국, 디자이너, 일군, 전통, 통해, 발전, 문화, 산업, 사실...
4655635                             [폴리스스토리, 시리즈, 부터, 하나, 최고]
9251303                              [연기, 진짜, 생각, 몰입, 진짜, 영화]
10067386                                   [안개, 밤하늘, 초승달, 영화]
                                  ...                        
8963373                                             [포켓, 몬스터]
3302770                                                    []
5458175                       [완전, 사이코, 영화, 마지막, 더욱더, 영화, 린다]
6908648                                       [라따뚜이, 스머프, 런가]
8548411                                      [저그, 영차, 영차, 영차]
Name: nouns, Length: 2000, dtype: object

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit_transform(df3.nouns) #리스트가 아니라 문자열로 넣어줘야함

TypeError: ignored

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(df3.nouns.apply(" ".join)) #리스트가 아니라 문자열로 넣어줘야함
print(x) 

  (0, 1206)	1
  (0, 2670)	1
  (0, 551)	1
  (1, 722)	1
  (1, 3120)	1
  (1, 2059)	1
  (1, 721)	1
  (1, 2309)	1
  (1, 2479)	2
  (1, 2972)	1
  (1, 1119)	1
  (1, 1011)	1
  (1, 1363)	1
  (1, 1345)	1
  (1, 2090)	1
  (1, 1640)	1
  (1, 1964)	1
  (1, 494)	1
  (1, 1333)	1
  (2, 3067)	1
  (2, 1628)	1
  (2, 1266)	1
  (2, 3098)	1
  (2, 2814)	1
  (3, 1945)	1
  :	:
  (1990, 2892)	1
  (1990, 859)	1
  (1990, 629)	1
  (1991, 1977)	1
  (1991, 1957)	1
  (1991, 67)	1
  (1991, 2347)	1
  (1991, 435)	1
  (1992, 2947)	1
  (1993, 1990)	1
  (1994, 102)	1
  (1994, 1202)	1
  (1995, 3057)	1
  (1995, 963)	1
  (1997, 1977)	2
  (1997, 2044)	1
  (1997, 847)	1
  (1997, 1353)	1
  (1997, 625)	1
  (1997, 819)	1
  (1998, 766)	1
  (1998, 1578)	1
  (1998, 746)	1
  (1999, 2447)	1
  (1999, 1976)	3


In [27]:
cv.vocabulary_ # 사전의 구성

{'보고': 1206,
 '지금': 2670,
 '다시': 551,
 '디자인': 722,
 '학생': 3120,
 '외국': 2059,
 '디자이너': 721,
 '일군': 2309,
 '전통': 2479,
 '통해': 2972,
 '발전': 1119,
 '문화': 1011,
 '산업': 1363,
 '사실': 1345,
 '우리나라': 2090,
 '시절': 1640,
 '열정': 1964,
 '노라노': 494,
 '사람': 1333,
 '폴리스스토리': 3067,
 '시리즈': 1628,
 '부터': 1266,
 '하나': 3098,
 '최고': 2814,
 '연기': 1945,
 '진짜': 2711,
 '생각': 1404,
 '몰입': 964,
 '영화': 1977,
 '안개': 1770,
 '밤하늘': 1124,
 '초승달': 2806,
 '사랑': 1336,
 '라면': 747,
 '처음': 2772,
 '완전': 2044,
 '감동': 52,
 '전쟁': 2475,
 '바보': 1067,
 '나이': 430,
 '훗날': 3280,
 '사하나': 1360,
 '감정': 62,
 '이해': 2273,
 '고질': 199,
 '오페라': 2024,
 '작품': 2381,
 '극단': 323,
 '갈림': 42,
 '반전': 1105,
 '평점': 3046,
 '긴장감': 374,
 '스릴': 1574,
 '전장': 2474,
 '공포': 219,
 '고시': 189,
 '소재': 1512,
 '관련': 236,
 '단연': 565,
 '가면': 10,
 '갈수록': 44,
 '더욱': 624,
 '밀회': 1061,
 '화이팅': 3243,
 '수작': 1549,
 '일본': 2315,
 '마음': 840,
 '임팩트': 2342,
 '일품': 2334,
 '제대로': 2541,
 '범죄': 1174,
 '스릴러': 1575,
 '마디': 827,
 '징텅': 2728,
 '교복': 249,
 '이의': 2256,
 '볼펜': 1241,
 '자국':

In [28]:
# 학습시켜서 가져오기
from sklearn.model_selection import train_test_split

y = df3.label # 타깃, 종속변수 y
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [29]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_test, pred) # 원래 50%인데 개선이 된 것

0.678

## 데이터 처리와 관련된 코랩 데이터
https://colab.research.google.com/drive/1LPvMgmPFOfpWgQsck0t5yCIi97ncrM3f?usp=sharing
